In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import ConcatDataset
import torchvision
from torchvision import transforms

from dataloading.nvidia import NvidiaCropWide, Normalize, NvidiaTrainDataset, NvidiaValidationDataset
from network import PilotNet
from trainer import Trainer

import wandb

%load_ext autoreload
%autoreload 2

# Config

In [2]:
# Set path to where you bag files are extracted
#root_path = Path("/home/romet/data/datasets/ut/nvidia-data")
root_path = Path("/home/romet/data2/datasets/rally-estonia/dataset")
use_augmentation = False
model_name = f"models/1-pilotnet-test-v3"

# Hyperparameters
n_epochs = 100
learning_rate = 1e-3
weight_decay = 0.01

## Datasets

In [3]:
tr = transforms.Compose([NvidiaCropWide(), Normalize()])
trainset = NvidiaTrainDataset(root_path)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False,
                                         num_workers=32, pin_memory=True, persistent_workers=True)

/home/romet/data2/datasets/rally-estonia/dataset/2021-05-20-12-36-10_e2e_sulaoja_20_30: lenght=12025, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-05-20-12-43-17_e2e_sulaoja_20_30: lenght=6809, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-05-20-12-51-29_e2e_sulaoja_20_30: lenght=5393, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-05-20-13-44-06_e2e_sulaoja_10_10: lenght=3833, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-05-20-13-51-21_e2e_sulaoja_10_10: lenght=3798, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-05-20-13-59-00_e2e_sulaoja_10_10: lenght=687, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-05-28-15-07-56_e2e_sulaoja_20_30: lenght=15626, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-05-28-15-17-19_e2e_sulaoja_20_30: lenght=3218, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-06-09-13-14-51_e2e_rec_ss2: lenght=49918, filtere

In [4]:
validset = NvidiaValidationDataset(root_path)
validloader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=False,
                                         num_workers=32, pin_memory=True, persistent_workers=True)

/home/romet/data2/datasets/rally-estonia/dataset/2021-05-28-15-19-48_e2e_sulaoja_20_30: lenght=10708, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-06-07-14-20-07_e2e_rec_ss6: lenght=25836, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-06-07-14-06-31_e2e_rec_ss6: lenght=3003, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-06-07-14-09-18_e2e_rec_ss6: lenght=4551, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-06-07-14-36-16_e2e_rec_ss6: lenght=25368, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-09-24-14-03-45_e2e_rec_ss11_backwards: lenght=25172, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-10-26-10-49-06_e2e_rec_ss20_elva: lenght=33045, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: lenght=33281, filtered=0
/home/romet/data2/datasets/rally-estonia/dataset/2021-10-20-15-11-29_e2e_rec_vastse_ss13_17_back: lenght=26763, 

## Train model

In [5]:
model = PilotNet()
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), 
                              eps=1e-08, weight_decay=weight_decay, amsgrad=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

trainer = Trainer(model_name, target_name='steering_angle', wandb_logging=False)
trainer.train(model, trainloader, validloader, optimizer, criterion, n_epochs)

  0%|          | 0/2065 [00:00<?, ?it/s]

  0%|          | 0/348 [00:00<?, ?it/s]

Saving best model.


  0%|          | 0/2065 [00:00<?, ?it/s]

KeyboardInterrupt: 